### Spark Initialization

In [1]:
import findspark
findspark.init()

In [2]:
import sys
import copy
import csv

from string import atoi
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
import numpy as np

In [4]:
conf = SparkConf().setAppName("ContentBased")
conf = conf.setMaster("local[*]")

In [5]:
sc  = SparkContext(conf=conf)

### Load Train and Test Data

In [6]:
trainData = sc.textFile("/Users/lakshya/Desktop/INF-553/Project/pittsburgh_review_with_text_20_res_lemma_data_train.txt",use_unicode=False)
testData = sc.textFile("/Users/lakshya/Desktop/INF-553/Project/pittsburgh_review_with_text_20_res_lemma_data_test.txt",use_unicode=False)

In [7]:
train_rdd = trainData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), float(x[2])))
test_rdd = testData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), float(x[2])))

In [8]:
avg_rating = train_rdd.map(lambda x: (x[0][0], x[1])).groupByKey().map(lambda x: (x[0], list(x[1]))).map(lambda x: (x[0], sum(x[1])/len(x[1])))
prod_rating = train_rdd.map(lambda x: (x[0][1], x[1])).groupByKey().map(lambda x: (x[0], list(x[1]))).map(lambda x: (x[0], sum(x[1])/len(x[1])))

In [9]:
train_temp = trainData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), 1))

### Load review data

In [10]:
data = sc.textFile("/Users/lakshya/Desktop/INF-553/Project/pittsburgh_review_with_text_20_res_lemma_data_train.txt",use_unicode=False)

In [11]:
data.take(6)

['1VVHf1BvtGC0aSCCIjQyiA,K5jY2W5Q3eNnwssV5UZtow,4,2016-11-16,2,2,2,past sunday one several time ive spirit its always eccentric fun time first music performance second hang recent visit sundays bingo bango spirit know pizza good drink also awesome last time get chard margarita time get tomatillo margarita hot ciders its always pleasant surprise see whats menu food drink drink little pricey drawback one coolest things spirit atmosphere its always super strange positive way really never know expect bingo bango definitely family appropriate its fun activity friends maybe even date doesnt mind something ordinary',
 'QYKexxaOJQlseGWmc6soRg,rzByiKaj-bLeLz-zKNBQdg,2,2015-04-13,0,0,0,old cramp build lot enough employees staff keep demand cause long wait time',
 '-ARdx8hOcEWlMDjzwLYZ_g,3cbsPfoUUrysf-M8FI_0IA,4,2014-03-24,6,4,3,live long world without donut menu dont know group nine din three varieties donuts include lemon lavender chocolate espresso zeppolli amaze pepper donut concoction ever m

In [12]:
train_data = data.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), (x[2],x[7]))).join(train_temp)

In [13]:
train_data.take(3)

[(('3egcdazws_x1wW35jgXfNw', 'gae9LAyt7Qvf_OgAkWASxA'),
  (('4',
    'family always love buca di beppo location food great service always excellent however wait time food delivery final bill keep us remember come return experience buca di beppo wonderful love family style din make reservations years occasion price isnt bad receive well discount coupons everywhere look know say family return'),
   1)),
 (('4wp4XI9AxKNqJima-xahlg', '7O_mNtg37-1sMvQ5xmA8Dw'),
  (('4',
    'appreciate whole vibe old reclaim wood chocolate color wall beautiful crystal chandeliers almost like shouldnt go together go well hear serve lunch want check along something sweet course go roast beef sandwich fresh load good stuff definitely need mayo thats totally give know condiment obsession love bun wasnt hard bite downstairs like sweet treat wonderland glitter lot glitter know theres edible glitter its home base im totally okay past glitter delicious look treat every nook corner cake cookies bon bons even ice cre

### Collect user data from train data (User, Product, Rating)

In [14]:
userReview = train_data.map(lambda x: (x[0][0], x[0][1], x[1][0][0]))

In [15]:
userReviewCollected = userReview.map(lambda x: (x[0], x[1], x[2]))

In [16]:
userReviewCollected.take(5)

[('3egcdazws_x1wW35jgXfNw', 'gae9LAyt7Qvf_OgAkWASxA', '4'),
 ('4wp4XI9AxKNqJima-xahlg', '7O_mNtg37-1sMvQ5xmA8Dw', '4'),
 ('BxDsaVNeWxc5mNyA1HtSHQ', 'oeW0vIYd3rUnAPgmD4fEFg', '5'),
 ('BBg_86FEejn3dNzj0JOR9Q', 'XItYW5ul3OW_AqpT2nDbBQ', '4'),
 ('z2YVGKKcup6mjQmDQ6arEg', 'RqmORv3974ZDC6Zh4nSQwg', '5')]

### Collect product data (Product, Review Text)

In [17]:
prodReview = train_data.map(lambda x: (x[0][1], x[1][0][1])).groupByKey().mapValues(list)

In [18]:
prodReviewCollected = prodReview.map(lambda x: (x[0], " ".join(x[1])))

In [19]:
prodReviewCollected.take(5)

[('5REYrZfsX3m4E3FTwovp5Q',
  'try first time last night pretty good one thing really annoy restaurant see review verde entire restaurant fill reservations can not even seat walk 2 people restaurant empty really avocado corn ceviche mojo criollo nigiri avocado crab delicious mojo criollo favorite everything taste super fresh flavorful serve good size date share wine meh would think place thats like tapas wine bar wine would better order frontera sauvignon blanc often buy liquor store slo wine hm lame hate pay 9 glass wine restaurant easily buy 10 liquor store seem justify least slo bottle service good nice time would come back absolutely love tacos havent try anything else menu taco addiction simply put food delicious however portion extremely small order leave place full youll spend pretty penny wouldnt recommend place youre big eater would terrific spot go show light snack drink food nicely present tapas style really nothing remarkable dont go dinner often would impress presentation 

### TF-IDF vector creation

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

/Users/lakshya/anaconda/lib/python2.7/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [21]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1,1),min_df=0, norm='l2')

Convert user and product rdd to pandas dataframe

In [22]:
spark = SparkSession(sc)

userPandas = userReviewCollected.toDF().toPandas()
prodPandas = prodReviewCollected.toDF().toPandas()

Create TF-IDF vectors on product review text

In [23]:
tfidf_prod = tf.fit_transform(prodPandas['_2'])

In [24]:
tfidf_prod.shape

(3098, 26685)

Add TF-IDF vectors to product dataframe

In [25]:
prodPandas['Vector'] = tfidf_prod.toarray().tolist()

In [26]:
prodPandas

,_1,_2,Vector
0,5REYrZfsX3m4E3FTwovp5Q,try first time last night pretty good one thin...,"[0.0048004686407, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,HWrbZS1mxVRj2Y2VwMmDMg,oh man word can not describe excite bakeshop i...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,MvlQo4bev1eqp1q0HYOLHg,first dance class probably decade probably hav...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,X9Bql7RrPU5Mab5-hJsI8A,2nd visit promise first time order feature bur...,"[0.0224731737297, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,owO2UkNKk9qrWWd_PTYLDA,feel like ive random experience place regulars...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,rc9SfYli96cnlPDMyJ7hyA,go lunch friends group 7 round table back nice...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,94VxE6XLTrXmz4yXWEJbAg,come instead ds need buy case beer theyre chea...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,anfgvbiobw4M0Wnvx_os3Q,its good place try ive never wow pretty much d...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,vLGYQ96AjKV5zpW0q3795g,breakfast pizza checkgood pizza accent employe...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,7l-CCyNnvT86GMR5EtYjog,jamaican tacos tonight die cant speak anything...,"[0.0, 0.0147491129158, 0.0, 0.0, 0.0, 0.0, 0.0..."


Product category text not needed

In [27]:
del prodPandas['_2']

### Weighted Linear Combination of product vectors for users

In [28]:
userPandas['Vector'] = [[] for _ in range(len(userPandas))]

In [29]:
userPandas['_3'] = userPandas['_3'].astype(float)

In [30]:
for index, row in userPandas.iterrows():
    vector = np.array(prodPandas.loc[prodPandas['_1'] == row['_2'], 'Vector'].values[0])
    rating = row['_3']
    userPandas.at[index,'Vector'] = rating*vector

In [31]:
userPandas

,_1,_2,_3,Vector
0,3egcdazws_x1wW35jgXfNw,gae9LAyt7Qvf_OgAkWASxA,4.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,4wp4XI9AxKNqJima-xahlg,7O_mNtg37-1sMvQ5xmA8Dw,4.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,BxDsaVNeWxc5mNyA1HtSHQ,oeW0vIYd3rUnAPgmD4fEFg,5.0,"[0.0444835542224, 0.0264260403666, 0.0, 0.0, 0..."
3,BBg_86FEejn3dNzj0JOR9Q,XItYW5ul3OW_AqpT2nDbBQ,4.0,"[0.022492077401, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,z2YVGKKcup6mjQmDQ6arEg,RqmORv3974ZDC6Zh4nSQwg,5.0,"[0.0783406184889, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
5,kb36ZHxGLdPU5p63Mj5u4w,mBycrsguRPhI0xtFBzsz_w,4.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,TkFcPIHDPx43OqnqZ_ElPQ,SmkYLXEYhzwUZdS6TAevHg,4.0,"[0.0186145886613, 0.0165873617779, 0.0, 0.0, 0..."
7,cUUCMncwWZm57s95iUX1Gg,dW79jPJVpzlTKGF-1JAsaw,3.0,"[0.0206199481973, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
8,JVJOfFqPmZEvkRznzF8w-w,yZ_wpRr4Svw8YRZ9voCgIw,4.0,"[0.048556663323, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
9,Q0Bs5ObN4w2U5WHs2gLsPA,MsFj3NTvmhYNYS8Ef8HwFA,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Product and rating column not needed

In [32]:
del userPandas['_2']
del userPandas['_3']

Linear combination of feature vectors

In [33]:
userPandas = userPandas.groupby(['_1']).sum()

Normalize the user feature vectors

In [34]:
from sklearn.preprocessing import Normalizer

In [35]:
for index, row in userPandas.iterrows():
    vector = np.array(row['Vector']).reshape(1, -1)
    transformer = Normalizer().fit(vector)
    userPandas.at[index,'Vector'] = transformer.transform(vector)

In [36]:
userPandas

,Vector
_1,
-0-hVEpwWEcJLJoGq3rE3g,"[[0.00707135105417, 0.000361628795416, 0.0, 0...."
-2OB54nQ6FsGLUM-R1KXnA,"[[0.0081805485151, 0.000732556578107, 0.0, 0.0..."
-ARdx8hOcEWlMDjzwLYZ_g,"[[0.00891272247392, 0.00101404014352, 0.0, 0.0..."
-Pk25bOBsvemFaWKDBVBzA,"[[0.00582838200541, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
-Q2wBtscwW6JOqlBndji4A,"[[0.00646852989731, 0.0, 0.0, 0.0, 0.001247097..."
-Q4bjWlbxmb1yKP4U7OODg,"[[0.0066520987401, 0.00109638791695, 0.0, 0.0,..."
-SDx-d5jppC4OBBosLVpYw,"[[0.0120166382623, 0.00245985137857, 0.0, 0.0,..."
-XgVXGJnOnW0kQEol6O3Pg,"[[0.00432558973424, 0.0, 0.0, 0.00055252360114..."
-Y6tXYPYqeVy37-L5p0rMw,"[[0.007849066616, 0.00144409264297, 0.0, 0.0, ..."


### Create user numpy matrix from feature vectors

In [37]:
user_matrix = np.zeros((len(userPandas), tfidf_prod.shape[1]))
idx = 0
for index, row in userPandas.iterrows():
    vector = np.array(row['Vector'])[0]
    user_matrix[idx] = vector
    idx += 1
    

In [38]:
user_matrix

array([[ 0.00707135,  0.00036163,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00818055,  0.00073256,  0.        , ...,  0.        ,
         0.        ,  0.00038655],
       [ 0.00891272,  0.00101404,  0.        , ...,  0.        ,
         0.0003156 ,  0.        ],
       ..., 
       [ 0.0110992 ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.0069594 ,  0.00076676,  0.        , ...,  0.        ,
         0.        ,  0.0004189 ],
       [ 0.0060828 ,  0.00035974,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

### Create product numpy matrix from feature vectors

In [39]:
prod_matrix = np.zeros((len(prodPandas), tfidf_prod.shape[1]))
idx = 0
for index, row in prodPandas.iterrows():
    vector = np.array(row['Vector'])
    prod_matrix[idx] = vector
    idx += 1

In [40]:
prod_matrix

array([[ 0.00480047,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

### Compute cosine similarity by taking dot product

In [41]:
similarity_matrix = np.dot(user_matrix, prod_matrix.T)

In [42]:
similarity_matrix.shape

(987, 3098)

### Flatten similarity matrix to relate with user and products

In [43]:
prod = prodPandas['_1'].values
user = userPandas.index.values

In [44]:
zf = similarity_matrix.flatten()
xr = np.repeat(user, prod.size)
yt = np.tile(prod, user.size)
d = np.stack((xr, yt, zf), axis=-1)

In [45]:
d.shape

(3057726, 3)

### Convert similarity matrix to RDD

In [46]:
similarity_rdd = sc.parallelize(d,10000)

In [47]:
similarity_rdd.take(5)

[array([u'-0-hVEpwWEcJLJoGq3rE3g', u'5REYrZfsX3m4E3FTwovp5Q',
        0.47567194032152077], dtype=object),
 array([u'-0-hVEpwWEcJLJoGq3rE3g', u'HWrbZS1mxVRj2Y2VwMmDMg',
        0.18905336787947438], dtype=object),
 array([u'-0-hVEpwWEcJLJoGq3rE3g', u'MvlQo4bev1eqp1q0HYOLHg',
        0.13765104745595919], dtype=object),
 array([u'-0-hVEpwWEcJLJoGq3rE3g', u'X9Bql7RrPU5Mab5-hJsI8A',
        0.40540105082096445], dtype=object),
 array([u'-0-hVEpwWEcJLJoGq3rE3g', u'owO2UkNKk9qrWWd_PTYLDA',
        0.22314458296277775], dtype=object)]

### Use Train and Test Data for predictions

In [48]:
trainData.take(5)

['1VVHf1BvtGC0aSCCIjQyiA,K5jY2W5Q3eNnwssV5UZtow,4,2016-11-16,2,2,2,past sunday one several time ive spirit its always eccentric fun time first music performance second hang recent visit sundays bingo bango spirit know pizza good drink also awesome last time get chard margarita time get tomatillo margarita hot ciders its always pleasant surprise see whats menu food drink drink little pricey drawback one coolest things spirit atmosphere its always super strange positive way really never know expect bingo bango definitely family appropriate its fun activity friends maybe even date doesnt mind something ordinary',
 'QYKexxaOJQlseGWmc6soRg,rzByiKaj-bLeLz-zKNBQdg,2,2015-04-13,0,0,0,old cramp build lot enough employees staff keep demand cause long wait time',
 '-ARdx8hOcEWlMDjzwLYZ_g,3cbsPfoUUrysf-M8FI_0IA,4,2014-03-24,6,4,3,live long world without donut menu dont know group nine din three varieties donuts include lemon lavender chocolate espresso zeppolli amaze pepper donut concoction ever m

Create key on (user, product)

In [49]:
train_rdd = trainData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), float(x[2])))
test_rdd = testData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), float(x[2])))

In [50]:
similarity_rdd = similarity_rdd.map(lambda x: ((x[0], x[1]), float(x[2])))

In [51]:
similarity_rdd.take(5)

[((u'-0-hVEpwWEcJLJoGq3rE3g', u'5REYrZfsX3m4E3FTwovp5Q'), 0.47567194032152077),
 ((u'-0-hVEpwWEcJLJoGq3rE3g', u'HWrbZS1mxVRj2Y2VwMmDMg'), 0.18905336787947438),
 ((u'-0-hVEpwWEcJLJoGq3rE3g', u'MvlQo4bev1eqp1q0HYOLHg'), 0.13765104745595919),
 ((u'-0-hVEpwWEcJLJoGq3rE3g', u'X9Bql7RrPU5Mab5-hJsI8A'), 0.40540105082096445),
 ((u'-0-hVEpwWEcJLJoGq3rE3g', u'owO2UkNKk9qrWWd_PTYLDA'), 0.22314458296277775)]

Join similarity matrix with train and test RDD to take only similarity values for training and testing

In [52]:
train = similarity_rdd.join(train_rdd)
test = similarity_rdd.join(test_rdd)

In [53]:
train.take(5)

[((u'cCke3VtFLtqgzkgYSrSE2g', u'BMR_AsSBzTQHqW-SQabI4w'),
  (0.5177639053023504, 4.0)),
 ((u'YE54kKTuqJJPNYWIKIpOEQ', u'RvwZqjdkZ_pER0moPXLZAQ'),
  (0.6235338140983966, 3.0)),
 ((u'4wp4XI9AxKNqJima-xahlg', u'ziJsGjXvidzZWC1I0-SOSg'),
  (0.6145243735413134, 3.0)),
 ((u'U92V-gp13uMZL-sl_naTHA', u'zi6cB_bkswWPLD2k3IVtyg'),
  (0.5046606382779443, 4.0)),
 ((u'rZQCd47n7OwPd71igVX6Og', u'IfHkboQZGPZkKpqHZCW4ag'),
  (0.37659631391587844, 1.0))]

Convert RDD to List

In [54]:
train_ratings = train.collect()
test_ratings = test.collect()

In [55]:
test_rdd.take(5)

[(('JiPMk9WmbJu-VfTRAKpZpw', 'PdDpIGwBZoTYzOVasT-WuA'), 4.0),
 (('2wKnvn68eWybc7ID-7UQmQ', 'khRo2a5OaIjumox-tkg3GA'), 4.0),
 (('LsWpfxWjLQcazDqnZ_A62g', 'D_pwairtGGR0V_w2xx5XeA'), 2.0),
 (('0N9bSCmoJMoGmR0EldzjQg', '3iaOYhNoc6XL935MqnxJSQ'), 5.0),
 (('-hietrA8M58asfpyJkCyiA', 'O1ird5yRyuDFnOmYu90OoA'), 4.0)]

In [56]:
train_ratings

[((u'cCke3VtFLtqgzkgYSrSE2g', u'BMR_AsSBzTQHqW-SQabI4w'),
  (0.5177639053023504, 4.0)),
 ((u'YE54kKTuqJJPNYWIKIpOEQ', u'RvwZqjdkZ_pER0moPXLZAQ'),
  (0.6235338140983966, 3.0)),
 ((u'4wp4XI9AxKNqJima-xahlg', u'ziJsGjXvidzZWC1I0-SOSg'),
  (0.6145243735413134, 3.0)),
 ((u'U92V-gp13uMZL-sl_naTHA', u'zi6cB_bkswWPLD2k3IVtyg'),
  (0.5046606382779443, 4.0)),
 ((u'rZQCd47n7OwPd71igVX6Og', u'IfHkboQZGPZkKpqHZCW4ag'),
  (0.37659631391587844, 1.0)),
 ((u'3KkT6SmPFLGvBS1pnDBr8g', u'9DxJTxiTJQdWI85MahuWPg'),
  (0.43319582359643555, 4.0)),
 ((u'ciXjBfJrAEteIKpzZg4I9g', u'P4lfCI0WSaVoA2VFdsZRiQ'),
  (0.6932661766681724, 2.0)),
 ((u'UtkjXTiMfGRz4R1zDF3Kxg', u'kKFUNcLklJ54zZNUk-7yjQ'),
  (0.3838884641493544, 4.0)),
 ((u'DeXKbQYNx52OlOizobOLJw', u'i-2OzvZUDtvKCMq1vcRSZg'),
  (0.3330687594413177, 5.0)),
 ((u'EzVLcZ6QwLLh2oD2e_dnLg', u'hNCKIrMJSacjMiWVwRb_YQ'),
  (0.354108570621633, 4.0)),
 ((u'MDeA-Gx8NScCyAdziq7IeA', u'Z6_gRH_m8FQlKgsNkZKmTg'),
  (0.5331474558064767, 4.0)),
 ((u'SKre6GKJAA3FLgb2e875Rg', u

In [57]:
train.take(5)

[((u'cCke3VtFLtqgzkgYSrSE2g', u'BMR_AsSBzTQHqW-SQabI4w'),
  (0.5177639053023504, 4.0)),
 ((u'YE54kKTuqJJPNYWIKIpOEQ', u'RvwZqjdkZ_pER0moPXLZAQ'),
  (0.6235338140983966, 3.0)),
 ((u'4wp4XI9AxKNqJima-xahlg', u'ziJsGjXvidzZWC1I0-SOSg'),
  (0.6145243735413134, 3.0)),
 ((u'U92V-gp13uMZL-sl_naTHA', u'zi6cB_bkswWPLD2k3IVtyg'),
  (0.5046606382779443, 4.0)),
 ((u'rZQCd47n7OwPd71igVX6Og', u'IfHkboQZGPZkKpqHZCW4ag'),
  (0.37659631391587844, 1.0))]

### Training Regressor on similarity values

In [58]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

Convert data to numpy array for regressor

In [59]:
X_train = []
y_train = []
for ratings in train_ratings:
    X_train.append(ratings[1][0])
    y_train.append(ratings[1][1])

X_train = np.array(X_train)
X_train = X_train.reshape(-1,1)

In [60]:
X_test = []
y_test = []
for ratings in test_ratings:
    X_test.append(ratings[1][0])
    y_test.append(ratings[1][1])

X_test = np.array(X_test)
X_test = X_test.reshape(-1,1)

In [61]:
X_test.shape

(9890, 1)

Train regressor on training data and make prediction on Test data<br>
Computed Mean Squared Error on predicted values

In [64]:
forest = RandomForestRegressor(max_depth=1, n_estimators=50)

rs = GradientBoostingRegressor(loss='ls', learning_rate=0.005, n_estimators=20)

reg = LinearRegression()

rs.fit(X_train, y_train)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

expected = y_test
predicted = rs.predict(X_test)

train_expected = y_train
train_predicted = rs.predict(X_train)

print("Training:\n%s" % np.sqrt(mean_squared_error(train_expected, train_predicted)))
print("Result:\n%s" % np.sqrt(mean_squared_error(expected, predicted)))

Training:
1.03137059321
Result:
1.03775086639


### Computed Ratings on missing values

In [65]:
missing_test = test_rdd.subtractByKey(test)

In [66]:
missing_ratings_user = missing_test.map(lambda x: ((x[0][0]), (x[0][1], x[1]))).join(avg_rating).map(lambda x: ((x[0], x[1][0][0]), (x[1][1], x[1][0][1])))

In [67]:
missing_ratings_user.take(5)

[(('5JVY32_bmTBfIGpCCsnAfw', 'ym7uH6ZpCfKQ1Of7eaYGCA'),
  (3.7881773399014778, 5.0)),
 (('5JVY32_bmTBfIGpCCsnAfw', '8c2ObImjXJ9pW0hUccL35A'),
  (3.7881773399014778, 4.0)),
 (('EpZFiJxo6RO8a6QwDwC7TQ', 'AA-FZmy6JmzaAoavIUTaUQ'),
  (3.7708333333333335, 5.0)),
 (('hg71CGHy9bwpgY8cGVVg4w', 'gB7X5o33o6W_rAGgHunSCA'),
  (3.707317073170732, 3.0)),
 (('WoKCLSctS7G2547xKcED-Q', '8A4ck_lvMLYZDrDt7w8GVA'),
  (3.9285714285714284, 2.0))]

In [71]:
predictions = test.map(lambda x: ((x[0]), (rs.predict(np.array(x[1][0]).reshape(1,-1))[0], x[1][1])))

In [72]:
final_predictions = predictions.union(missing_ratings_user)

In [73]:
final_predictions.count()

10002

In [74]:
mse = final_predictions.map(lambda x: (x[1][0]-x[1][1])**2)

In [75]:
np.sqrt(mse.mean())

1.0382404089064161

In [76]:
final_predictions = final_predictions.collect()

### Save predictions file

In [77]:
with open('/Users/lakshya/Desktop/INF-553/Project/TFIDF_ReviewBasedPredictions_pittsburgh.txt', 'w') as f:
    for item in final_predictions:
        f.write(str(item[0][0])+","+str(item[0][1])+","+str(item[1][0])+"\n")

### Making predictions on training data for Hybrid model

In [80]:
train_predict = train.map(lambda x: ((x[0]), (rs.predict(np.array(x[1][0]).reshape(1,-1))[0], x[1][1])))

In [81]:
train_predict.take(5)

[((u'cCke3VtFLtqgzkgYSrSE2g', u'BMR_AsSBzTQHqW-SQabI4w'),
  (3.8266873834325938, 4.0)),
 ((u'YE54kKTuqJJPNYWIKIpOEQ', u'RvwZqjdkZ_pER0moPXLZAQ'),
  (3.8386699757233549, 3.0)),
 ((u'4wp4XI9AxKNqJima-xahlg', u'ziJsGjXvidzZWC1I0-SOSg'),
  (3.8386699757233549, 3.0)),
 ((u'U92V-gp13uMZL-sl_naTHA', u'zi6cB_bkswWPLD2k3IVtyg'),
  (3.8266873834325938, 4.0)),
 ((u'rZQCd47n7OwPd71igVX6Og', u'IfHkboQZGPZkKpqHZCW4ag'),
  (3.8306061915479983, 1.0))]

In [82]:
train_predictions = train_predict.map(lambda x: (x[1][0]-x[1][1])**2)

In [83]:
np.sqrt(train_predictions.mean())

KeyboardInterrupt: 

In [84]:
train_predict = train_predict.collect()

In [169]:
with open('/Users/lakshya/Desktop/INF-553/Project/TrainTFIDF_ReviewBasedPredictions_pittsburgh.txt', 'w') as f:
    for item in train_predict:
        f.write(str(item[0][0])+","+str(item[0][1])+","+str(item[1][0])+"\n")